In [12]:
!pip install mysql-connector-python
!pip install pymysql
!pip uninstall -y langchain langchain-community langchain-core
!pip install langchain==0.1.20
!pip install langchain-community==0.0.36
!pip install langchain-core==0.1.52
!pip install langchain-text-splitters
!pip install transformers sentence-transformers faiss-cpu pypdf

Found existing installation: langchain 0.1.20
Uninstalling langchain-0.1.20:
  Successfully uninstalled langchain-0.1.20
Found existing installation: langchain-community 0.0.36
Uninstalling langchain-community-0.0.36:
  Successfully uninstalled langchain-community-0.0.36
Found existing installation: langchain-core 0.1.52
Uninstalling langchain-core-0.1.52:
  Successfully uninstalled langchain-core-0.1.52
  Using cached langchain-0.1.20-py3-none-any.whl.metadata (13 kB)
  Using cached langchain_community-0.0.38-py3-none-any.whl.metadata (8.7 kB)
  Using cached langchain_core-0.1.53-py3-none-any.whl.metadata (5.9 kB)
Using cached langchain-0.1.20-py3-none-any.whl (1.0 MB)
Using cached langchain_community-0.0.38-py3-none-any.whl (2.0 MB)
Using cached langchain_core-0.1.53-py3-none-any.whl (303 kB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-classic 1.0.0 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.1.53 which is incompatible.
langchain-classic 1.0.0 requires langchain-text-splitters<2.0.0,>=1.0.0, but you have langchain-text-splitters 0.0.2 which is incompatible.
langgraph-checkpoint 3.0.1 requires langchain-core>=0.2.38, but you have langchain-core 0.1.53 which is incompatible.
langgraph-prebuilt 1.0.5 requires langchain-core>=1.0.0, but you have langchain-core 0.1.53 which is incompatible.


  Using cached langchain_community-0.0.36-py3-none-any.whl.metadata (8.7 kB)
Using cached langchain_community-0.0.36-py3-none-any.whl (2.0 MB)
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.0.38
    Uninstalling langchain-community-0.0.38:
      Successfully uninstalled langchain-community-0.0.38


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.1.20 requires langchain-community<0.1,>=0.0.38, but you have langchain-community 0.0.36 which is incompatible.


  Using cached langchain_core-0.1.52-py3-none-any.whl.metadata (5.9 kB)
Using cached langchain_core-0.1.52-py3-none-any.whl (302 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.53
    Uninstalling langchain-core-0.1.53:
      Successfully uninstalled langchain-core-0.1.53


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.1.20 requires langchain-community<0.1,>=0.0.38, but you have langchain-community 0.0.36 which is incompatible.
langchain-classic 1.0.0 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.1.52 which is incompatible.
langchain-classic 1.0.0 requires langchain-text-splitters<2.0.0,>=1.0.0, but you have langchain-text-splitters 0.0.2 which is incompatible.
langgraph-checkpoint 3.0.1 requires langchain-core>=0.2.38, but you have langchain-core 0.1.52 which is incompatible.
langgraph-prebuilt 1.0.5 requires langchain-core>=1.0.0, but you have langchain-core 0.1.52 which is incompatible.


In [23]:
# Imports
import re
import pandas as pd
import mysql.connector
import logging

from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_community.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA

from transformers import pipeline
from transformers import logging as hf_logging
from langchain.prompts import PromptTemplate


from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

In [14]:
hf_logging.set_verbosity_error()
logging.basicConfig(level=logging.INFO)

In [15]:
# Data Loading
query = "SELECT * FROM shop_inventory_flat;"

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="S@i300498",
    database="shop_inventory"
)

df = pd.read_sql(query, conn)
conn.close()

df.head()


C:\Users\admine\AppData\Local\Temp\ipykernel_7156\2695973830.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,inventory_id,shop_name,product_name,quantity,product_type,location,day_in_to_shop,available,price
0,1,FreshMart,Milk,50,Amul 1L,"123 MG Road, Bangalore",2026-01-20,1,60.0
1,2,FreshMart,Bread,30,Britannia 400g,"123 MG Road, Bangalore",2026-01-15,1,40.0
2,3,FreshMart,Sugar,120,Madhur 1kg,"123 MG Road, Bangalore",2026-01-21,1,55.0
3,4,FreshMart,Banana,0,FreshFarms 1kg,"123 MG Road, Bangalore",2026-01-22,0,70.0
4,5,DailyNeeds,Eggs,20,LocalFarm 12pcs,"45 Park Street, Kolkata",2026-01-18,1,90.0


In [16]:
documents = []

for _, row in df.iterrows():
    text = f"""
    Product: {row['product_name']}
    Brand/Type: {row['product_type']}
    Shop Name: {row['shop_name']}
    Location: {row['location']}
    Available: {"Yes" if row['available'] == 1 else "No"}
    Quantity in Stock: {row['quantity']}
    Price: ₹{row['price']}
    Date Added: {row['day_in_to_shop']}
    """

    documents.append(
        Document(
            page_content=text.strip(),
            metadata={
                "inventory_id": row["inventory_id"]
            }
        )
    )

print(f"Documents Created: {len(documents)}")

Documents Created: 10


In [17]:
# Chunking

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50
)

documents_chunked = text_splitter.split_documents(documents)

print(f"Chunks Created: {len(documents_chunked)}")

Chunks Created: 10


In [18]:
# Embeddings
embedding_function = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


In [19]:
# Vector Store 

vectorstore = FAISS.from_documents(
    documents_chunked,
    embedding_function
)

retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)


In [20]:
# ==== LLM ====

llm_pipeline = pipeline(
    "text-generation",
    model="tiiuae/falcon-7b-instruct",
    device="cpu",
    max_new_tokens=150,
    temperature=0.3
)
llm = HuggingFacePipeline(pipeline=llm_pipeline)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [21]:
def extract_keywords(query):
    query = query.lower()
    words = re.findall(r"[a-zA-Z0-9]+", query)
    stopwords = {
        "where", "can", "i", "get", "find", "buy",
        "the", "a", "an", "is", "available"
    }
    return [w for w in words if w not in stopwords]

def dynamic_product_filter(docs, keywords):
    filtered = []
    for doc in docs:
        text = doc.page_content.lower()
        if any(keyword in text for keyword in keywords):
            filtered.append(doc)
    return filtered

In [24]:
   
#CHAT LOOP

while True:
    user_query = input("\nHow may I assist you (type 'exit' to quit): ")

    if user_query.lower() == "exit":
        print("👋 Exiting...")
        break

    #  Retrieve candidates
    retrieved_docs = retriever.get_relevant_documents(user_query)

    #  Extract keywords from query
    keywords = extract_keywords(user_query)

    #  Filter documents dynamically
    filtered_docs = dynamic_product_filter(retrieved_docs, keywords)

    if not filtered_docs:
        print("\nAnswer:")
        print("Product not available in inventory.")
        continue

    #  Build grounded context
    context = "\n\n".join(doc.page_content for doc in filtered_docs)

    # Strict prompt
    prompt = f"""
You are an inventory assistant.
Answer ONLY using the information provided below.
Do NOT use general knowledge.
Do NOT guess.

Context:
{context}

Question:
{user_query}

Answer in ONE sentence including:
Shop Name, Location, Price.
"""

    response = llm(prompt)

    print("\nAnswer:")
    print(response.strip())

    print("\nMatched Records:")
    for doc in filtered_docs:
        print("-" * 50)
        print(doc.page_content)



How may I assist you (type 'exit' to quit):  where can i get amul milk?


C:\Users\admine\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(



Answer:
You are an inventory assistant.
Answer ONLY using the information provided below.
Do NOT use general knowledge.
Do NOT guess.

Context:
Product: Milk
    Brand/Type: Amul 1L
    Shop Name: FreshMart
    Location: 123 MG Road, Bangalore
    Available: Yes
    Quantity in Stock: 50
    Price: ₹60.0
    Date Added: 2026-01-20

Question:
where can i get amul milk?

Answer in ONE sentence including:
Shop Name, Location, Price.

Answer:
Shop Name: FreshMart, Location: 123 MG Road, Bangalore, Price: ₹60.0.

Matched Records:
--------------------------------------------------
Product: Milk
    Brand/Type: Amul 1L
    Shop Name: FreshMart
    Location: 123 MG Road, Bangalore
    Available: Yes
    Quantity in Stock: 50
    Price: ₹60.0
    Date Added: 2026-01-20



How may I assist you (type 'exit' to quit):  exit


👋 Exiting...
